In [1]:
from napari_ome_zarr_navigator.ome_zarr_image import OMEZarrImage
import zarr
import napari
from devtools import debug


In [2]:
# path = "/Users/joel/Desktop/20200812-CardiomyocyteDifferentiation14-Cycle1_mip.zarr/B/03/0"
path = "/Users/joel/Documents/Code/napari-ome-zarr-navigator/test_data/10_5281_zenodo_10424292/20200812-CardiomyocyteDifferentiation14-Cycle1_mip.zarr/B/03/0"

image = OMEZarrImage(path)

In [3]:
image.get_table_attrs("measurements")

{'encoding-type': 'anndata',
 'encoding-version': '0.1.0',
 'fractal_table_version': '1',
 'instance_key': 'label',
 'region': {'path': '../labels/nuclei'},
 'type': 'feature_table'}

In [12]:
def get_omero_metadata(image, channel):
    for channel in image.image_meta.omero.channels:
        if channel.label == channel:
            return channel
    return None 

In [7]:
my_dict = {
    "label1": "some obj",
    "label2": "other ob"
}
list(my_dict.keys())[0]

'label1'

In [13]:
channels = get_omero_metadata(image, "DAPI")

In [16]:
for channel in channels:
    print(type(channel))

<class 'fractal_tasks_core.ngff.specs.Channel'>
<class 'fractal_tasks_core.ngff.specs.Channel'>
<class 'fractal_tasks_core.ngff.specs.Channel'>


[Channel(window=Window(max=65535.0, min=0.0, start=110.0, end=800.0), label='DAPI', family=None, color='00FFFF', active=True),
 Channel(window=Window(max=65535.0, min=0.0, start=110.0, end=250.0), label='nanog', family=None, color='FF00FF', active=True),
 Channel(window=Window(max=65535.0, min=0.0, start=110.0, end=800.0), label='Lamin B1', family=None, color='FFFF00', active=True)]

In [21]:
img, scale = image.load_intensity_roi(
        roi_of_interest = 0, 
        channel_index = 0,
        level="0",
        roi_table="FOV_ROI_table",
    )

In [22]:
roi_table = "FOV_ROI_table"
roi_table = "new_masking_ROI_table"
roi_table = "nuclei_ROI_table"
roi_an = image.read_table(roi_table)

In [23]:
roi_an.obs.index

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '1483', '1484', '1485', '1486', '1487', '1488', '1489', '1490', '1491',
       '1492'],
      dtype='object', length=1493)

: 

In [5]:
viewer = napari.Viewer()
viewer.add_image(img, scale=scale)

<Image layer 'img' at 0x28a34b0d0>

In [11]:
image.image_meta.multiscales[0].axes

[Axis(name='c', type='channel', unit=None),
 Axis(name='z', type='space', unit='micrometer'),
 Axis(name='y', type='space', unit='micrometer'),
 Axis(name='x', type='space', unit='micrometer')]

In [10]:
image.image_meta.multiscales[0].axes

image.image_meta.multiscales[0].axes[-1].name == "x"


True

In [6]:
image.image_meta.multiscales[0].datasets

Dataset(path='0', coordinateTransformations=[ScaleCoordinateTransformation(type='scale', scale=[1.0, 1.0, 0.1625, 0.1625])])

In [7]:
datasets = image.image_meta.multiscales[0].datasets

In [10]:
datasets

[Dataset(path='0', coordinateTransformations=[ScaleCoordinateTransformation(type='scale', scale=[1.0, 1.0, 0.1625, 0.1625])]),
 Dataset(path='1', coordinateTransformations=[ScaleCoordinateTransformation(type='scale', scale=[1.0, 1.0, 0.325, 0.325])]),
 Dataset(path='2', coordinateTransformations=[ScaleCoordinateTransformation(type='scale', scale=[1.0, 1.0, 0.65, 0.65])]),
 Dataset(path='3', coordinateTransformations=[ScaleCoordinateTransformation(type='scale', scale=[1.0, 1.0, 1.3, 1.3])]),
 Dataset(path='4', coordinateTransformations=[ScaleCoordinateTransformation(type='scale', scale=[1.0, 1.0, 2.6, 2.6])])]

In [5]:
print(image.get_tables_list())
print(image.get_tables_list(table_type="ROIs"))
print(image.get_tables_list(table_type="feature_table"))

['FOV_ROI_table', 'new_masking_ROI_table', 'nuclei', 'well_ROI_table']
['FOV_ROI_table', 'new_masking_ROI_table', 'well_ROI_table']
['nuclei']


In [3]:
channel_names = []

for channel in image.image_meta.omero.channels:
    channel_names.append(channel.label)

In [4]:
image.get_pyramid_levels()

['0', '1', '2', '3', '4']

In [9]:
with zarr.open(image.zarr_url, mode="r") as zarr_group:
    sub_groups = [name for name in zarr_group.group_keys()]
    print(zarr_group.labels)
print(sub_groups)

<zarr.hierarchy.Group '/labels' read-only>
['labels', 'tables']


In [4]:
image.get_labels_list()

['manual_label_correction_6', 'new_label_img_1', 'nuclei']

In [6]:
image.get_tables_list()

['FOV_ROI_table', 'new_masking_ROI_table', 'nuclei', 'well_ROI_table']

In [8]:
my_selection = "labels"
with zarr.open(image.zarr_url, mode="r") as zarr_group:
    print(zarr_group[my_selection])

<zarr.hierarchy.Group '/labels' read-only>


In [7]:
with zarr.open(image.zarr_url, mode="r") as zarr_group:
    sub_groups = list(zarr_group.group_keys())
    print(zarr_group.labels)
print(sub_groups)

<zarr.hierarchy.Group '/labels' read-only>
['labels', 'tables']


In [20]:
from pathlib import Path
import anndata as ad

def read_table(zarr_url: Path, roi_table):
    with zarr.open(zarr_url, mode="r").tables[
                roi_table
            ] as table:
        return ad.read_zarr(table)
    # TODO: Make this work for cloud-based files => different paths?
    # table_url = zarr_url / f"tables/{roi_table}"
    # return ad.read_zarr(table_url)

In [22]:
path = "/Users/joel/Desktop/20200812-CardiomyocyteDifferentiation14-Cycle1_mip.zarr/B/03/0"
roi_table = "FOV_ROI_table"
read_table(path, roi_table).to_df()
# read_table(Path(path), roi_table).to_df()

<zarr.hierarchy.Group '/tables/FOV_ROI_table' read-only>


,x_micrometer,y_micrometer,z_micrometer,len_x_micrometer,len_y_micrometer,len_z_micrometer,x_micrometer_original,y_micrometer_original
FieldIndex,,,,,,,,
FOV_1,0.0,0.0,0.0,416.0,351.0,1.0,-1448.300049,-1517.699951
FOV_2,416.0,0.0,0.0,416.0,351.0,1.0,-1032.300049,-1517.699951
